![logo](images/untumbes.PNG)

<center><b>Prof. Dr. Jorge Zavaleta - zavaleta.jorge@gmail.com</b></center>

# Algoritmo FP-Growth

In [1]:
#librarys
import pandas as pd
import numpy as np
import random
#
import heapq
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
#
from surprise import Dataset, Reader, KNNBasic
# ml
from surprise.model_selection import train_test_split

# graphics
import matplotlib.pylab as plt
%matplotlib inline
#
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Passo 1: Gerar um conjunto de dados aleatório
data = [['Leite', 'Cebola', 'Noz-Moscada', 'Feijão', 'Ovos', 'Iogurte'],
        ['Dill', 'Cebola', 'Noz-Moscada', 'Feijão', 'Ovos', 'Iogurte'],
        ['Leite', 'Maçã', 'Feijão', 'Ovos'],
        ['Leite', 'Milho', 'Noz-Moscada', 'Feijão', 'Iogurte'],
        ['Milho', 'Cebola', 'Cebola', 'Feijão', 'Sorvete', 'Ovos']]
data

[['Leite', 'Cebola', 'Noz-Moscada', 'Feijão', 'Ovos', 'Iogurte'],
 ['Dill', 'Cebola', 'Noz-Moscada', 'Feijão', 'Ovos', 'Iogurte'],
 ['Leite', 'Maçã', 'Feijão', 'Ovos'],
 ['Leite', 'Milho', 'Noz-Moscada', 'Feijão', 'Iogurte'],
 ['Milho', 'Cebola', 'Cebola', 'Feijão', 'Sorvete', 'Ovos']]

In [3]:
# Passo 2: Pré-processamento dos dados
te = TransactionEncoder()
te_tmp = te.fit(data).transform(data)
df = pd.DataFrame(te_tmp, columns=te.columns_)
df

,Cebola,Dill,Feijão,Iogurte,Leite,Maçã,Milho,Noz-Moscada,Ovos,Sorvete
0,True,False,True,True,True,False,False,True,True,False
1,True,True,True,True,False,False,False,True,True,False
2,False,False,True,False,True,True,False,False,True,False
3,False,False,True,True,True,False,True,True,False,False
4,True,False,True,False,False,False,True,False,True,True


In [4]:
df.shape

(5, 10)

In [5]:
# Passo 3: Aplicar o algoritmo FP-Growth
frequent_itemsets = fpgrowth(df, min_support=0.6, use_colnames=True)
frequent_itemsets[['itemsets','support']]

,itemsets,support
0,(Feijão),1.0
1,(Ovos),0.8
2,(Noz-Moscada),0.6
3,(Leite),0.6
4,(Iogurte),0.6
5,(Cebola),0.6
6,"(Ovos, Feijão)",0.8
7,"(Feijão, Noz-Moscada)",0.6
8,"(Leite, Feijão)",0.6
9,"(Noz-Moscada, Iogurte)",0.6


In [6]:
# Collecting the inferred rules in a dataframe 
reglas = association_rules(frequent_itemsets, metric ="lift", min_threshold = 1) 
reglas = reglas.sort_values(['confidence', 'lift'], ascending =[False, False]) 
reglas.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
6,(Noz-Moscada),(Iogurte),0.6,0.6,0.6,1.0,1.666667,0.24,inf,1.0
7,(Iogurte),(Noz-Moscada),0.6,0.6,0.6,1.0,1.666667,0.24,inf,1.0
10,"(Feijão, Noz-Moscada)",(Iogurte),0.6,0.6,0.6,1.0,1.666667,0.24,inf,1.0
11,"(Feijão, Iogurte)",(Noz-Moscada),0.6,0.6,0.6,1.0,1.666667,0.24,inf,1.0
14,(Noz-Moscada),"(Feijão, Iogurte)",0.6,0.6,0.6,1.0,1.666667,0.24,inf,1.0
15,(Iogurte),"(Feijão, Noz-Moscada)",0.6,0.6,0.6,1.0,1.666667,0.24,inf,1.0
16,(Cebola),(Ovos),0.6,0.8,0.6,1.0,1.250000,0.12,inf,0.5
21,"(Cebola, Feijão)",(Ovos),0.6,0.8,0.6,1.0,1.250000,0.12,inf,0.5
23,(Cebola),"(Ovos, Feijão)",0.6,0.8,0.6,1.0,1.250000,0.12,inf,0.5
0,(Ovos),(Feijão),0.8,1.0,0.8,1.0,1.000000,0.00,inf,0.0


>## Análisis

Podemos interpretar las reglas resultantes de la siguiente manera:

Valor de soporte -> muestra que quien compra Noz-Moscada compra Iogurte tiene un soporte de 0.6

Valor de Confianza -> muestra que el 100% de los clientes que compran Noz-Moscada también compran Iogurte.

Valor de elevación -> muestra que las ventas de (Iogurte) aumentan en 1.67 para las compras con Noz-Moscada.

>## Cesta do Tio José

In [16]:
# Load and preprocess data set
ctjose = pd.read_csv('data/cesta.csv', sep=';', encoding='latin-1')
ctjose.set_index('TID', inplace=True)
ctjose.head(10)

,Café,Desinfetante,Carne suina,Detergente,Limpador multiuso,Limpa pisos de madeira,Frango,agua,Esponjas,Limpa-vidros,...,Baldes,Tomates,Leite,alcool,Panos de chao,Carne bovina,Cha,Bananas,Suco de laranja,Carne de peixe
TID,,,,,,,,,,,,,,,,,,,,,
0,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,1,0,1
1,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,1,0,1
2,0,1,0,1,0,1,0,1,0,1,...,1,0,1,0,1,0,1,0,1,0
3,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,1,0,1
4,0,1,0,1,0,1,0,1,0,1,...,1,0,1,0,1,0,1,0,1,0
5,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,1,0,1
6,0,1,0,1,0,1,0,1,0,1,...,1,0,1,0,1,0,1,0,0,1
7,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,1,0,1
8,0,1,0,1,0,1,0,1,0,1,...,1,0,1,0,1,0,1,0,1,0


In [17]:
ctjose.shape

(23, 25)

In [20]:
# Passo 2: Pré-processamento dos dados- ya está en 1/0
#te_tj = TransactionEncoder()
#te_tj_tmp = te_tj.fit(ctjose).transform(ctjose)
#data_tj = pd.DataFrame(te_tj_tmp, columns=te_tj.columns_)
#data_tj.head()

In [24]:
# aplicando el algoritmo
cesta_items = fpgrowth(ctjose, min_support=0.5, use_colnames=True)
cesta_items[['itemsets','support']]

,itemsets,support
0,(Carne de peixe),0.608696
1,(Bananas),0.565217
2,(Carne bovina),0.565217
3,(alcool),0.521739
4,(Tomates),0.521739
...,...,...
8186,"(Sabao neutro ou de coco, Laranjas, Tomates, C...",0.521739
8187,"(Sabao neutro ou de coco, Laranjas, Tomates, C...",0.521739
8188,"(Sabao neutro ou de coco, Laranjas, Tomates, C...",0.521739
8189,"(Sabao neutro ou de coco, Laranjas, Tomates, a...",0.521739


In [25]:
# Collecting the inferred rules in a dataframe 
tjose_rules = association_rules(cesta_items, metric ="confidence", min_threshold = 0.5) 
tjose_rules = tjose_rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
tjose_rules.head(10) 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
66,(alcool),(Tomates),0.521739,0.521739,0.521739,1.0,1.916667,0.249527,inf,1.0
67,(Tomates),(alcool),0.521739,0.521739,0.521739,1.0,1.916667,0.249527,inf,1.0
68,(alcool),(Laranjas),0.521739,0.521739,0.521739,1.0,1.916667,0.249527,inf,1.0
69,(Laranjas),(alcool),0.521739,0.521739,0.521739,1.0,1.916667,0.249527,inf,1.0
70,(alcool),(Macas),0.521739,0.521739,0.521739,1.0,1.916667,0.249527,inf,1.0
71,(Macas),(alcool),0.521739,0.521739,0.521739,1.0,1.916667,0.249527,inf,1.0
72,(alcool),(Sabao neutro ou de coco),0.521739,0.521739,0.521739,1.0,1.916667,0.249527,inf,1.0
73,(Sabao neutro ou de coco),(alcool),0.521739,0.521739,0.521739,1.0,1.916667,0.249527,inf,1.0
74,(alcool),(Esponjas),0.521739,0.521739,0.521739,1.0,1.916667,0.249527,inf,1.0
75,(Esponjas),(alcool),0.521739,0.521739,0.521739,1.0,1.916667,0.249527,inf,1.0


## Análisis

Podemos interpretar las reglas resultantes de la siguiente manera:

Valor de soporte -> muestra que quien compra Álcool compra tomates tiene un soporte de 0.52

Valor de Confianza -> muestra que el 100% de los clientes que compran {Álcool,Tomates,Laranjas,Macas} también compran {Tomates,alcool,laranjas, Macas}.

Valor de elevación -> muestra que las ventas de (Tomates) aumentan en 1.92 para las compras con Alcool.

---
<center><b>&copy;Jorge Zavaleta, 2024</b></center>